In [1]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("openlm-research/open_llama_3b_v2")
model = AutoModelForCausalLM.from_pretrained("openlm-research/open_llama_3b_v2")

/Users/isaac/FundamentlPartners/abinvenv-sol/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


In [2]:
prompt = 'Q: What is the largest animal?\nA:'
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

generation_output = model.generate(
    input_ids=input_ids, max_new_tokens=32
)
print(tokenizer.decode(generation_output[0]))

<s> Q: What is the largest animal?
A: The largest animal is the blue whale.
Q: What is the smallest animal?
A: The smallest animal is the dwarf chameleon


In [3]:
from transformers import pipeline
hf_pipeline = pipeline(task="text-generation", model=model, tokenizer=tokenizer)

In [4]:
from langchain.llms import HuggingFaceEndpoint
from langchain import HuggingFacePipeline

from dotenv import load_dotenv
from langchain.vectorstores import Pinecone
from langchain.chains import RetrievalQAWithSourcesChain, RetrievalQA
from langchain.prompts import PromptTemplate

#utils
# from src.utils import connect_index

# Memory

load_dotenv()

import os
os.getcwd()

import sys 

sys.path.append(os.path.abspath(os.path.join('../')))

from config import *
from src.utils import connect_index
from langchain.vectorstores import Pinecone

/Users/isaac/FundamentlPartners/abinvenv-sol


In [5]:
embedding_multi_qa = InferenceEndpointHuggingFaceEmbeddings("https://aet52zbntx9zffq4.us-east-1.aws.endpoints.huggingface.cloud", HUGGING_FACE_API_TOKEN)
index = connect_index("stab-test")
vector_store = Pinecone(index, embedding_multi_qa.embed_query, "text")


WARNING 2023-08-17 16:06:47,119 [root:__init__.py:_maybe_print_use_warning:468]

  command:

    streamlit run /Users/isaac/FundamentlPartners/abinvenv-sol/venv/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]



2023-08-17 16:06:47.119 
  command:

    streamlit run /Users/isaac/FundamentlPartners/abinvenv-sol/venv/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]


INFO 2023-08-17 16:06:47,836 [src.utils:utils.py:connect_index:13]
Connected to Pinecone index stab-test



2023-08-17 16:06:47.836 Connected to Pinecone index stab-test
/Users/isaac/FundamentlPartners/abinvenv-sol/venv/lib/python3.10/site-packages/langchain/vectorstores/pinecone.py:61: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


In [6]:
llm = HuggingFacePipeline(pipeline=hf_pipeline) 

In [20]:
from langchain.prompts import PromptTemplate
prompt_template = """ 
ANSWER THE QUESTION: {question}
GIVEN THE CONTEXT: {summaries}
ANSWER: 
"""
prompt = PromptTemplate(
    template=prompt_template, input_variables=["summaries", "question"]
)

chain_type_kwargs = {"prompt": prompt}

In [21]:
qa_chain = RetrievalQAWithSourcesChain.from_chain_type(
                                llm=llm, 
                                chain_type="stuff", 
                                retriever=vector_store.as_retriever(),
                                reduce_k_below_max_tokens=True,
                                return_source_documents=True,
                                chain_type_kwargs=chain_type_kwargs)

In [22]:
qa_chain({"question": "What is Blockchain?"})


INFO 2023-08-17 16:24:36,028 [src.models.HuggingFaceEmbeddings:HuggingFaceEmbeddings.py:embed_query:30]
Querying Hugging Face API for What is Blockchain? inputs:



2023-08-17 16:24:36.028 Querying Hugging Face API for What is Blockchain? inputs:
Input length of input_ids is 2689, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


{'question': 'What is Blockchain?',
 'answer': '\n',
 'sources': '',
 'source_documents': [Document(page_content='process of recording transactions and tracking assets in a business network. An asset can be tangible (a house, a car, cash, land) or intangible (intellectual property, patents, copy- rights, branding). Virtually anything of value can be tracked and traded on a blockchain network, reducing risk and cutting costs for all involved.\n\nThat’s the elevator speech for blockchain. In the rest of this chap- ter, you review additional details to help you more fully appre- ciate this technology and its potential for streamlining business operations.\n\nTracing Blockchain’s Origin\n\nYou can gain a deeper understanding of blockchain by exploring the context in which it was developed: the need for an efficient, cost-effective, reliable, and secure system for conducting and recording financial transactions. In this section, I provide that context and describe the characteristics of blo